In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.metrics import DistanceMetric

# Other methods that were considered, but none of them offer a Haversine metric option
# from sklearn.neighbors import KDTree
# from scipy.spatial import KDTree
# import pykdtree

STARBUCKS_LOCATIONS = 'data/starbucks_us.csv'
STARBUCKS_COLS = ['starbucksid', 'lat', 'lon']
STARBUCKS_LATLONG_COLS = ['store_lat', 'store_long']
STARBUCKS_RADIANS_COLS = ['sb_lat_radians', 'sb_long_radians']
STARBUCKS_INDEX = 'starbucks_id'
STARBUCKS_MAP = {
    'starbucksid': 'starbucks_id', 
    'lat': 'store_lat', 
    'lon': 'store_long'
}

HOUSING_LOCATIONS = 'data/multifamily_physical_inspection_scores_0321.csv'
HOUSING_COLS = ['PROPERTY_ID', 'LATITUDE', 'LONGITUDE']
HOUSING_LATLONG_COLS = ['latitude', 'longitude']
HOUSING_RADIANS_COLS = ['lat_radians', 'long_radians']
HOUSING_INDEX = 'property_id'
# 3959 is the radius of the earth in miles, multiply to convert radians to miles
RADIANS_TO_MILES = 3959


In [2]:
def load_data(path: str, columns: list) -> pd.DataFrame:
    df = pd.read_csv(path, usecols=columns)
    return df


def latlong_to_radians(df: pd.DataFrame, new_cols: list, old_cols: list):
    df[new_cols] = (np.radians(df.loc[:, old_cols]))
    df.drop(columns=old_cols, inplace=True)
    return df


def downcast_fields(df: pd.DataFrame):
    # convert float64's to float32's to save space
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col]=pd.to_numeric(df[col], downcast='float')
    return df

In [3]:
sb_df = load_data(STARBUCKS_LOCATIONS, STARBUCKS_COLS)
h_df = load_data(HOUSING_LOCATIONS, HOUSING_COLS)

In [4]:
# format columns
sb_df.rename(columns=STARBUCKS_MAP, inplace=True)
h_df.columns = [col.lower() for col in h_df.columns]

# drop columns with no lat / long info
h_df.dropna(axis=0, how="any", subset=HOUSING_LATLONG_COLS, inplace=True)
sb_df.dropna(axis=0, how="any", subset=STARBUCKS_LATLONG_COLS, inplace=True)

In [5]:
h_df = latlong_to_radians(
    h_df, 
    HOUSING_RADIANS_COLS, 
    HOUSING_LATLONG_COLS
)

sb_df = latlong_to_radians(
    sb_df,
    STARBUCKS_RADIANS_COLS,
    STARBUCKS_LATLONG_COLS
)

h_df = downcast_fields(h_df)
sb_df = downcast_fields(sb_df)

In [6]:
# US starbucks locations
print(f'Total Starbucks Locations: {len(sb_df)}')
sb_df.head()

Total Starbucks Locations: 13620


,starbucks_id,sb_lat_radians,sb_long_radians
0,10357,0.829225,-1.615289
1,1006159,0.786278,-1.624230
2,1008940,0.790982,-1.633004
3,14598,0.784349,-1.625686
4,12449,0.784884,-1.628003


In [7]:
# US housing dataset
print(f'Total Housing Locations: {len(h_df)}')
h_df.head()

Total Housing Locations: 27415


,property_id,lat_radians,long_radians
0,800006929,0.713483,-1.492496
1,800005604,0.748410,-1.962701
2,800001529,0.594720,-2.059074
3,800027853,0.758258,-1.335069
4,800225287,0.673112,-1.608806


### Brute force method to find nearest starbucks to each house

In [8]:
def nearest_locations_pairwise(df1, df2):
    """
    Brute force method to find the nearest point to each point in two arrays
    It runs at On^2...

    df1: the dataframe you want to append the nearest starbucks to
    df2: the dataframe with all starbucks locations
    """
    # create references for better naming in this example
    houses_df = df1
    starbucks_df = df2    

    # use Haversine distance since earth is sphere
    dist = DistanceMetric.get_metric('haversine')

    dist_matrix = dist.pairwise(
        houses_df[HOUSING_RADIANS_COLS],
        starbucks_df[STARBUCKS_RADIANS_COLS]
    ) * RADIANS_TO_MILES

    # resultant df is a matrix of distances between all houses and starbucks stores
    distances_df = pd.DataFrame(
        dist_matrix, 
        index=houses_df[HOUSING_INDEX], 
        columns=starbucks_df[STARBUCKS_INDEX]
    )
    
    # Unpivot above dataframe from wide format to long format.
    distances_df = pd.melt(distances_df.reset_index(), id_vars=HOUSING_INDEX)
    
    # When you unpivot, the data in the pivot table becomes a column named 'value'. 
    # Rename this column to 'miles' for clarity.
    distances_df.rename(columns={'value':'miles'}, inplace=True)
    
    # Sort all location pairs by distance, keep the closest distance.
    distances_df.sort_values(['property_id', 'miles'], ascending=[False, True], inplace=True)
    distances_df.drop_duplicates(subset=[HOUSING_INDEX], keep='first', inplace=True)
    distances_df.reset_index(drop=True, inplace=True)
    
    # Merge df with distance pairs back into original df
    merged_df = houses_df.merge(distances_df, on=HOUSING_INDEX)
    merged_df.rename(columns={'miles': 'closest_starbucks (mi)'}, inplace=True)
    
    return merged_df

In [9]:
%%time 
# current fastest runtime 20:24
pairwise_df = nearest_locations_pairwise(h_df, sb_df)

In [ ]:
# Avg distance to a starbucks nationwide = 8.15 miles
print(f'Avg distance to a starbucks nationwide: {pairwise_df["closest_starbucks (mi)"].mean()}')
pairwise_df.head()

### Ball tree method to find nearest starbucks to each house

In [15]:
def nearest_locations_balltree(df1, df2):
    # create references for better naming in this example
    houses_df = df1
    starbucks_df = df2

    # Construct ball tree with starbucks coordinates
    ball = BallTree(starbucks_df[STARBUCKS_RADIANS_COLS].values, metric='haversine')

    # Execute query with the starbucks locations
    dist, idx = ball.query(houses_df[HOUSING_RADIANS_COLS].values, k=1)
    # convert to miles x 3959
    dist = dist * RADIANS_TO_MILES

    # get indices of nearest starbucks and merge with housing
    starbucks_filtered = starbucks_df[[STARBUCKS_INDEX]]
    sb_nearest = starbucks_filtered.iloc[np.squeeze(idx)]
    
    houses_filtered = houses_df.drop(columns=['lat_radians', 'long_radians'])
    final_df = pd.concat([
        houses_filtered.reset_index(drop=True), 
        sb_nearest.reset_index(drop=True), 
        pd.Series(np.squeeze(dist), name='closest_starbucks (mi)')
    ], axis=1)

    return final_df

In [16]:
%%time
# current fastest runtime 00:01.44
balltree_df = nearest_locations_balltree(h_df, sb_df)

CPU times: user 2.07 s, sys: 7.62 ms, total: 2.08 s
Wall time: 2.08 s


In [17]:
# Avg distance to a starbucks nationwide = 8.15mi
print(f'Avg distance to a starbucks nationwide: {balltree_df["closest_starbucks (mi)"].mean()}')
balltree_df.head()

Avg distance to a starbucks nationwide: 8.149461896565596


,property_id,starbucks_id,closest_starbucks (mi)
0,800006929,1009517,17.003990
1,800005604,7436,0.636020
2,800001529,1015032,0.360383
3,800027853,13542,25.169123
4,800225287,8388,2.160373


In [14]:
# Check to make sure result are same between methods
(pairwise_df['closest_starbucks (mi)'] != balltree_df['closest_starbucks (mi)']).any()